In [1]:
from pathlib import Path

import catboost as cb
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
data_path = Path("../data/")
seed = 42

In [3]:
def balanced_log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    n0, n1 = np.bincount(y_true.astype(int))
    w0 = 1 / (n0 / len(y_true))
    w1 = 1 / (n1 / len(y_true))

    l0 = -w0 / n0 * np.sum(np.where(y_true == 0, 1, 0) * np.log(1 - y_pred))
    l1 = -w1 / n1 * np.sum(np.where(y_true != 0, 1, 0) * np.log(y_pred))

    return (l0 + l1) / (w0 + w1)


In [4]:
# read data
train_df = pd.read_csv(data_path / "train.csv")
test_df = pd.read_csv(data_path / "test.csv")
greeks_df = pd.read_csv(data_path / "greeks.csv")

In [5]:
# some columns have trailing spaces
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
feature_cols = train_df.columns.tolist()[1:-1]

In [6]:
# training
oof = np.zeros(len(train_df))
skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
test_probs = []

In [7]:
fold = 1
for train_idx, val_idx in skf.split(train_df, greeks_df.iloc[:, 1:-1]):

    print(f"Fold-{fold}".center(110, "-"))
    fold += 1

    X_train, y_train = (
        train_df.loc[train_idx, feature_cols],
        train_df.loc[train_idx, "Class"],
    )

    X_val, y_val = (
        train_df.loc[val_idx, feature_cols],
        train_df.loc[val_idx, "Class"],
    )

    params = {
        "iterations": 10000,
        "early_stopping_rounds": 1000,
        "use_best_model": True,
        "learning_rate": 0.003,
        "l2_leaf_reg": 5,
        "random_strength": 2,
        "bagging_temperature": 1,
        "grow_policy": "Lossguide",
        "auto_class_weights": "Balanced",
    }

    model = cb.CatBoostClassifier(**params)
    model.fit(
        X_train, y_train, eval_set=[(X_val, y_val)], cat_features=["EJ"], verbose=1000
    )

    preds = model.predict_proba(X_val)[:, 1]
    oof[val_idx] = preds

    # make test predictions
    test_preds = model.predict_proba(test_df.iloc[:, 1:])
    test_probs.append(test_preds)

print(f"OOF score: {balanced_log_loss(train_df['Class'], oof):.4f}".center(110, "-"))


----------------------------------------------------Fold-1----------------------------------------------------
0:	learn: 0.6907571	test: 0.6915745	best: 0.6915745 (0)	total: 60.1ms	remaining: 10m 1s
1000:	learn: 0.1488318	test: 0.3976301	best: 0.3976120 (991)	total: 4.76s	remaining: 42.8s
2000:	learn: 0.0576062	test: 0.4071771	best: 0.3930983 (1362)	total: 9.46s	remaining: 37.8s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.3930983075
bestIteration = 1362

Shrink model to first 1363 iterations.
----------------------------------------------------Fold-2----------------------------------------------------
0:	learn: 0.6911850	test: 0.6917433	best: 0.6917433 (0)	total: 3.93ms	remaining: 39.3s
1000:	learn: 0.1618796	test: 0.3120733	best: 0.3120733 (1000)	total: 4.62s	remaining: 41.6s
2000:	learn: 0.0639922	test: 0.2568124	best: 0.2568124 (2000)	total: 9.61s	remaining: 38.4s
3000:	learn: 0.0201979	test: 0.2434941	best: 0.2416566 (2781)	total: 14.3s	remaining: 33.5s
St

In [8]:
# generate a submission file
test_probs = np.mean(test_probs, axis=0)
sub_df = pd.DataFrame(
    {"Id": test_df.Id, "Class_0": test_probs[:, 0], "Class_1": test_probs[:, 1]}
)
sub_df.to_csv("submission.csv", index=False)
